In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# %pip install CRPS

# from pathlib import Path
# import sys
# !sudo apt-get install unzip
# base = Path('/content/drive/MyDrive/ML/2023/conv_strat_dataset')
# sys.path.append(str(base))

# zip_path = base/"test.zip"


# !cp "{zip_path}" .

# !unzip -q test.zip -d "/content"

# !rm test.zip


In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import random
import json
from tqdm import tqdm
import os
from os import listdir
from PIL import Image

import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
from torch.optim import Adam
import datetime

# import wandb
# import CRPS.CRPS as pscore



from prec_dataset import RadarPrecipitationSequence
from radar_transforms import radar_transform
from plotting_funcs import show_sequence
from radar_transforms import radar_transform, reverse_transform
from fdp import fdp_sample, get_named_beta_schedule
from unet import UNet
from loss import loss_fn
from sampler import sample_next_step_pred
from helper_module import get_random_test_seq, get_CRPS_sequence
from calc_metrics import get_CRPS

config = dict(
    img_out_size=64,
    rgb_grayscale=1,
    sequence_length=8,
    max_prec_val=3.4199221045419974,
    prediction_time_step_ahead=1,
    frames_to_predict=1,
    num_cond_frames=4,
    tot_pred_ahead=4,
    schedule="linear",
    T=300,

    root_dir=r"/content/",
    validate_on_convective = False,
)
plot_folder = '/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min'
betas = get_named_beta_schedule(
    schedule_name=config["schedule"], num_diffusion_timesteps=config["T"]
)
T = config["T"]
if config["schedule"] == "linear":
    alphas = 1.0 - betas
    alphas_cumprod = torch.cumprod(alphas, axis=0)
    alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
    sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
    sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)
    posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)


device = "cuda" if torch.cuda.is_available() else "cpu"
model_PATH = '/content/drive/MyDrive/ML/results/plots/0503_concat/2023-03-05_epoch_95'

### LOAD PRE-TRAINED MODELS: 

In [ ]:
model = UNet(
    rgb_grayscale=config["rgb_grayscale"], num_cond_frames=config["num_cond_frames"], device=device
)
model.load_state_dict(torch.load(model_PATH,map_location=torch.device('cpu')))

model.eval()
model.to(device)




In [ ]:
dataset_20min_nowcast = RadarPrecipitationSequence(
    root_dir=config["root_dir"],
    transform=radar_transform(max_prec_val=config["max_prec_val"]),
    num_cond_frames=config["num_cond_frames"],
    frames_to_predict=config["frames_to_predict"],
    img_out_size=config["img_out_size"],
    prediction_time_step_ahead=config["prediction_time_step_ahead"],
    train_test_val = "test",
    center_crop = True,
    nowcast_mode = True
)


sample_idx_arr = np.load('/content/sample_idx_arr.npy')
sample_idx_arr = sample_idx_arr[:50]
sample_size = len(sample_idx_arr)
for smp in sample_idx_arr:
    print(f'{smp},')


# for i in sample_idx_arr:
#     train_sample = dataset_20min_nowcast.__getitem__(i)
    
#     seq = train_sample[0]
#     fig, axs = plt.subplots(1, 8, figsize=(20, 5))
#     fig.suptitle(f'{i}', fontsize=16)

#     ax1 = plt.subplot(181)
#     ax1.imshow((seq[0, :, :]), cmap="jet")

#     ax2 = plt.subplot(182)
#     ax2.imshow((seq[1, :, :]), cmap="jet")

#     ax3 = plt.subplot(183)
#     ax3.imshow((seq[2, :, :]), cmap="jet")

#     ax4 = plt.subplot(184)
#     im4 = ax4.imshow((seq[3, :, :]), cmap="jet")

#     ax4 = plt.subplot(185)
#     im4 = ax4.imshow((seq[4, :, :]), cmap="jet")

#     ax4 = plt.subplot(186)
#     im4 = ax4.imshow((seq[5, :, :]), cmap="jet")

#     ax4 = plt.subplot(187)
#     im4 = ax4.imshow((seq[6, :, :]), cmap="jet")

#     ax4 = plt.subplot(188)
#     im4 = ax4.imshow((seq[7, :, :]), cmap="jet")








#     fig.colorbar(im4)
    plt.show()



#### Metrics:

In [ ]:
from calc_metrics import get_nowcast_20_CRPS

num_of_samples = 10


crps_matrix= torch.zeros((4,4,sample_size))

preds_csi = torch.zeros((sample_size,4 ,64,64))
obs_csi = torch.zeros((sample_size,4,64,64))
viz_prob_forcast = torch.zeros((sample_size,num_of_samples,config["tot_pred_ahead"], 64,64)) 

with torch.no_grad():



    for sample_step,j in enumerate(tqdm((sample_idx_arr))):
        sample = dataset_20min_nowcast.__getitem__(j)
        lwe = sample[0]

        viz_prob_sample = torch.zeros((num_of_samples,config["tot_pred_ahead"], 64,64)) 
        
        obs_csi[sample_step,:,:,:] = lwe[4:,:,:]
        prev_frames = 4
        tot_pred_ahead = config["tot_pred_ahead"]

        pred_seq = torch.zeros((8,64,64))
        pred_seq[:prev_frames,:,:] = lwe[:prev_frames,:,:]  

        for i in range(prev_frames,prev_frames+config["tot_pred_ahead"]):
            #get 4 past frames:
            cond_frames = pred_seq[i-prev_frames:i, :, :]

            observed_nxt_step = lwe[i, :, :]

            cond_frames = cond_frames.to(device)
            observed_nxt_step = observed_nxt_step.to(device)

            cond_frames = cond_frames[None ,:,:]
            observed_nxt_step = observed_nxt_step[None,None,:,:]

           
            #pred next frame n times and get crps:
            avg_4_crps, avg_16_crps, max_4_crps, max_16_crps, preds_to_visualize = get_nowcast_20_CRPS(
                cond_frames = cond_frames,
                observation = observed_nxt_step, 
                sequence_length = config['sequence_length'],
                max_prec_val = config['max_prec_val'],
                betas = betas,
                rgb_grayscale=1,
                img_out_size=64,
                device=device,
                sqrt_one_minus_alphas_cumprod=sqrt_one_minus_alphas_cumprod,
                sqrt_recip_alphas=sqrt_recip_alphas,
                posterior_variance=posterior_variance,
                model=model,
                numb_of_samples = num_of_samples,
                T = 300,
            )

            viz_prob_sample[0, i-prev_frames, :, :] = preds_to_visualize[0]
            viz_prob_sample[1, i-prev_frames, :, :] = preds_to_visualize[1]
            viz_prob_sample[2, i-prev_frames, :, :] = preds_to_visualize[2]
            viz_prob_sample[3, i-prev_frames, :, :] = preds_to_visualize[3]

            crps_matrix[i-prev_frames, 0, sample_step] = avg_4_crps
            crps_matrix[i-prev_frames, 1, sample_step] = avg_16_crps
            crps_matrix[i-prev_frames, 2, sample_step] = max_4_crps
            crps_matrix[i-prev_frames, 3, sample_step] = max_16_crps

            
            
            
            # get next image in sequence;
            pred_next_frame = sample_next_step_pred(
            cond_frames = cond_frames,
            device = device,
            sqrt_one_minus_alphas_cumprod = sqrt_one_minus_alphas_cumprod,
            sqrt_recip_alphas = sqrt_recip_alphas,
            posterior_variance = posterior_variance,
            model = model,
            T = 300,
            betas = betas,
            rgb_grayscale=1,
            img_out_size=64)

            pred_seq[i,:,:] = pred_next_frame

        viz_prob_forcast[sample_step,:,:,:,:] = viz_prob_sample
        preds_csi[sample_step,:,:,:] = pred_seq[config['num_cond_frames']:,:,:]


print(preds_csi.max())
print(obs_csi.max())
print(crps_matrix.max())

preds_csi

torch.save(viz_prob_forcast, '/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/viz_prob_forcast.pt')


preds_csi_arr = preds_csi.numpy()
obs_csi_arr = obs_csi.numpy()
crps_matrix_arr = crps_matrix.numpy()

np.save('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/preds_csi.npy', preds_csi_arr)
np.save('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/obs_csi.npy', obs_csi_arr)
np.save('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/crps_matrix.npy', crps_matrix_arr)


In [ ]:
# plot_folder = "/content/drive/MyDrive/ML/results/nowcasting_metrics/0603_concat_basic"


# preds_csi_arr = preds_csi.numpy()
# obs_csi_arr = obs_csi.numpy()

# np.savetxt(f"{plot_folder}/preds_csi_arr.csv", preds_csi_arr, delimiter=",")
# np.savetxt(f"{plot_folder}/obs_csi_arr.csv", obs_csi_arr, delimiter=",")

### PLOT CRPS:


In [ ]:

time_steps = 4
pool_types = 4
crps_mean  = torch.zeros((pool_types,time_steps))
crps_std  = torch.zeros((pool_types,time_steps))

for ts in range(4):
    crps_i_min =  crps_matrix[ts,:,:]
    for pt in range(4):

        crps_pool = crps_i_min[pt,:]

        mean = crps_pool.mean()
        std = crps_pool.std()
        crps_mean[pt,ts] = mean 
        crps_std[pt,ts] = std 




print(crps_mean[:,0])
print(crps_std[:,0])



fig, axs = plt.subplots(1, 1, figsize=(25, 7))

ax1 = plt.subplot(121)
ticks = np.arange(5, 21, 5)
ax1.plot(ticks, crps_mean[:,0],  color="khaki", label='Pooling scale = 4 km(Avg)')
ax1.plot(ticks, crps_mean[:,1], color="skyblue", label='Pooling scale = 16 km(Avg)')

# ax1.errorbar(ticks, crps_mean[:,0],crps_std[:,0],color="m", label='Pooling scale = 4 km')
# ax1.errorbar(ticks, crps_mean[:,1],crps_std[:,1],color="skyblue", label='Pooling scale = 16 km')
ax1.set_xlabel('Prediction interval (min)')
ax1.set_ylabel('Avg-pooled CRPS')
ax1.plot(ticks, crps_mean[:,2],color="lightpink", label='Pooling scale = 4 km(max)')
ax1.plot(ticks, crps_mean[:,3],color="mediumseagreen", label='Pooling scale = 16 km(max)')
ax1.set_ylim([0,4.5])
ax1.legend()
ax1.set_xticks(ticks)



plt.show()




In [ ]:
breakkk

### Get CSI metrics:


In [ ]:




#TRANSFORM VALUES TO PRECIPIATION VALS:
preds_csi_rev_transformed = np.load('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/preds_csi.npy')


preds_csi_rev_transformed += 1
preds_csi_rev_transformed /= 2
preds_csi_rev_transformed *= config['max_prec_val']
preds_csi_rev_transformed **= 3

obs_csi_rev_transformed = np.load('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/obs_csi.npy')

obs_csi_rev_transformed += 1
obs_csi_rev_transformed /= 2
obs_csi_rev_transformed *= config['max_prec_val']
obs_csi_rev_transformed **= 3



def get_csi(predictions,observed, t):
    
    flat_preds = predictions.flatten()
    flat_obs = observed.flatten()
    
    #get : greater or equal
    F_geq_t = np.where(flat_preds>=t,1,0)
    O_geq_t = np.where(flat_obs >= t,1,0)
    O_les_t = np.where(flat_obs < t, 1,0)
    F_les_t = np.where(flat_preds < t, 1,0)
    
    tp = np.sum(np.logical_and(F_geq_t == 1, O_geq_t == 1))
    fp = np.sum(np.logical_and(F_geq_t == 1, O_les_t == 1))
    fn = np.sum(np.logical_and(F_les_t == 1, O_geq_t == 1))

    tp_fp_fn = tp+fp+fn

    if tp_fp_fn ==0 or tp == 0:
        return 0
    else:
        csi = tp / (tp+fp+fn)
        return csi
    
time_steps = 4
csi_0_01 = torch.zeros((sample_size, time_steps))
csi_2 = torch.zeros((sample_size, time_steps))
csi_5 = torch.zeros((sample_size, time_steps))


for smpl in range(sample_size):
    pred = preds_csi_rev_transformed[smpl,:,:,:]
    obs = obs_csi_rev_transformed[smpl,:,:,:]

    for i in range(time_steps):
        csi_0_01[smpl,i] = get_csi(predictions=pred[i], observed = obs[i], t=0.01)
        csi_2[smpl,i] = get_csi(predictions=pred[i], observed = obs[i], t=2.0)
        csi_5[smpl,i] = get_csi(predictions=pred[i], observed = obs[i], t=5)



def plot_csi(time_steps, csi_tensor):
    csi_mean = []
    csi_std = []

    for i in range(time_steps):
        csi_ = csi_tensor[:,i]

        csi_mean.append(csi_.mean())
        csi_std.append(csi_.std())
    return csi_mean, csi_std




csi_05_mean, csi_05_std = plot_csi(4,csi_0_01)
csi_2_mean, csi_2_std = plot_csi(4,csi_2)
csi_10_mean, csi_10_std = plot_csi(4,csi_5)

fig, axs = plt.subplots(1, 1, figsize=(15, 5))

ax1 = plt.subplot(111)
ticks = np.arange(5, 21, 5)
ax1.plot(ticks, csi_05_mean,marker = 'o',color="skyblue", label='Threshold: 0.01 mm/hr')
ax1.plot(ticks, csi_2_mean,marker = 'o',color="khaki", label='Threshold: 2 mm/hr')
ax1.plot(ticks, csi_10_mean,marker = 'o',color="pink", label='Threshold: 10 mm/hr')

ax1.set_xticks(ticks)
ax1.set_ylim([0,0.7])
ax1.set_xlabel('Prediction interval (min)')
ax1.set_ylabel('CSI')
ax1.legend()

print(csi_05_std)
print(csi_2_std)
print(csi_10_std)


### VISUALIZE 20 MIN FORECAST:

In [ ]:



def plot_sample_preds(viz_prob_forcast,obs, idx_arr):
    for viz_step,i in enumerate(tqdm((idx_arr))):
        seqs = viz_prob_forcast[viz_step]
        observed_seq = obs[(viz_step)]

        # vmax_plot = np.max(max_seq_val, max_obs_val)


        time = dataset_20min_nowcast.__getitem__(i)
        stamp_string = time[2]
        time_stamp = stamp_string.split(".")[0]
        time_stamp = int(time_stamp)
        stamp_arr = [time_stamp + (i * 60 * 5) for i in range(8)]

        timestamp_arr = [
        datetime.datetime.fromtimestamp(stamp).strftime("%Y-%m-%d %H:%M:%S")
        for stamp in stamp_arr
        ]
        timestamp_arr = timestamp_arr[config['num_cond_frames']:]






        fig, axs = plt.subplots(5, 4, figsize=(15, 20), sharex=True, sharey=True,  constrained_layout = True)


        axs[0,0].imshow(observed_seq[0,:,:], cmap="jet")#, vmin = 0)#, vmax = vmax_plot)
        axs[0,0].set_title(f"{timestamp_arr[0]}")
        for y_x in range(4):
            axs[y_x,0].set_ylabel('Km')
            axs[4,y_x].set_xlabel('Km')
        axs[4,0].set_ylabel('Km')

        axs[0,1].imshow(observed_seq[1,:,:], cmap="jet")#, vmin = 0)#, vmax = vmax_plot)
        axs[0,1].set_title(f"{timestamp_arr[1]}")

        axs[0,2].imshow(observed_seq[2,:,:], cmap="jet")#, vmin = 0)#, vmax = vmax_plot)
        axs[0,2].set_title(f"{timestamp_arr[2]}")

        im1 = axs[0,3].imshow(observed_seq[3,:,:], cmap="jet")#, vmin = 0)#, vmax = vmax_plot)
        axs[0,3].set_title(f"{timestamp_arr[3]}")
        cbar1 = fig.colorbar(im1, ax=axs[0, :], shrink= 0.75,location='right', pad = 0.01)  
        cbar1.set_label('Precipitation Rate (mm/hr)',fontsize=16)

        axs[1,0].imshow((seqs[0,0, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax1.set_title(f"{timestamp_arr[0]}")

        axs[1,1].imshow((seqs[0,1, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax2.set_title(f"{timestamp_arr[1]}")

        # ax3 = plt.subplot(143)
        axs[1,2].imshow((seqs[0,2, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax3.set_title(f"{timestamp_arr[2]}")

        # ax4 = plt.subplot(144)
        axs[1,3].imshow((seqs[0,3, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax4.set_title(f"{timestamp_arr[3]}")

        # ax1 = plt.subplot(141)
        axs[2,0].imshow((seqs[1,0, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax1.set_title(f"{timestamp_arr[0]}")

        # ax2 = plt.subplot(142)
        axs[2,1].imshow((seqs[1,1, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax2.set_title(f"{timestamp_arr[1]}")

        # ax3 = plt.subplot(143)
        axs[2,2].imshow((seqs[1,2, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax3.set_title(f"{timestamp_arr[2]}")

        # ax4 = plt.subplot(144)
        axs[2,3].imshow((seqs[1,3, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax4.set_title(f"{timestamp_arr[3]}")

        # ax1 = plt.subplot(141)
        axs[3,0].imshow((seqs[2,0, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax1.set_title(f"{timestamp_arr[0]}")

        # ax2 = plt.subplot(142)
        axs[3,1].imshow((seqs[2,1, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax2.set_title(f"{timestamp_arr[1]}")

        # ax3 = plt.subplot(143)
        axs[3,2].imshow((seqs[2,2, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax3.set_title(f"{timestamp_arr[2]}")

        # ax4 = plt.subplot(144)
        axs[3,3].imshow((seqs[2,3, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax4.set_title(f"{timestamp_arr[3]}")

        # ax1 = plt.subplot(141)
        axs[4,0].imshow((seqs[3,0, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)

        # ax1.set_title(f"{timestamp_arr[0]}")

        # ax2 = plt.subplot(142)
        axs[4,1].imshow((seqs[3,1, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax2.set_title(f"{timestamp_arr[1]}")

        # ax3 = plt.subplot(143)
        axs[4,2].imshow((seqs[3,2, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax3.set_title(f"{timestamp_arr[2]}")

        # ax4 = plt.subplot(144)
        axs[4,3].imshow((seqs[3,3, :, :]), cmap="jet")#, vmin = 0, vmax = vmax_plot)
        # ax4.set_title(f"{timestamp_arr[3]}")
        plt.show()

preds_csi_rev_transformed = torch.load('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/viz_prob_forcast.pt')
preds_csi_rev_transformed_copy = preds_csi_rev_transformed.clone().detach()



preds_csi_rev_transformed_copy += 1
preds_csi_rev_transformed_copy /= 2
preds_csi_rev_transformed_copy *= config['max_prec_val']
preds_csi_rev_transformed_copy **= 3

obs_csi_rev_transformed = np.load('/content/drive/MyDrive/ML/results/nowcasting_metrics/1003_20min/obs_csi.npy')

obs_csi_rev_transformed += 1
obs_csi_rev_transformed /= 2
obs_csi_rev_transformed *= config['max_prec_val']
obs_csi_rev_transformed **= 3

sample_idx_arr = np.load('/content/sample_idx_arr.npy')
sample_idx_arr = sample_idx_arr[:50]

plot_sample_preds(preds_csi_rev_transformed_copy, obs_csi_rev_transformed, sample_idx_arr)